In [1]:
import cifrum as lib
import pandas as pd
import numpy as np

## Asset Creation

The library can create an `asset` in several ways. The method `portfolio_asset` accepts parameters:

- either `name: str`, or `names: List[str]` of asset(s) to be created
- optional `start_period: str` and `end_period: str` to define date range. If nothing is provided then the library takes maximum possible range 
- optional `currency: str` to compute further activities. If no is provided then the library takes the default value for the financial symbol

In [2]:
asset_name = 'us/SPY'

In [3]:
def print_asset(asset):
    print('== {}'.format(asset.symbol.identifier_str))
    print(asset.currency)
    print(asset._period_min)

In [4]:
print_asset(lib.portfolio_asset(name=asset_name))

== us/SPY
Currency(USD)
1993-01


In [5]:
print_asset(lib.portfolio_asset(name=asset_name, start_period='2010-1'))

== us/SPY
Currency(USD)
2010-01


In [6]:
print_asset(lib.portfolio_asset(name=asset_name, start_period='2010-1', end_period='2013-1'))

== us/SPY
Currency(USD)
2010-01


In [7]:
asset=lib.portfolio_asset(name=asset_name, 
                        start_period='2010-1', end_period='2013-1', 
                        currency='usd')
print_asset(asset)

== us/SPY
Currency(USD)
2010-01


## Basic Activities

### Close

Every asset has the `close` values. The `close` method returns `TimeSeries` object that wraps `numpy` array with additional meta-information: `start_period`, `end_period`, and `TimeSeriesKind`

In [8]:
close = asset.close()
close

TimeSeries(start_period=2010-01, end_period=2013-01, kind=TimeSeriesKind.VALUES, values=[ 87.4115  90.1379  95.6258  97.1073  89.3891  84.7649  90.5544  86.4811
  94.2259  97.8259  97.8278 104.3657 106.7969 110.5068 110.5206 113.7215
 112.4461 110.5484 108.3372 102.3814  95.2737 105.6728 105.2433 106.3433
 111.2746 116.1045 119.8386 119.0387 111.8896 116.4352 117.8081 120.7595
 123.821  121.5679 122.26   123.3479 129.6624]

In [9]:
close.values

array([ 87.4115,  90.1379,  95.6258,  97.1073,  89.3891,  84.7649,
        90.5544,  86.4811,  94.2259,  97.8259,  97.8278, 104.3657,
       106.7969, 110.5068, 110.5206, 113.7215, 112.4461, 110.5484,
       108.3372, 102.3814,  95.2737, 105.6728, 105.2433, 106.3433,
       111.2746, 116.1045, 119.8386, 119.0387, 111.8896, 116.4352,
       117.8081, 120.7595, 123.821 , 121.5679, 122.26  , 123.3479,
       129.6624])

In [10]:
close.start_period, close.end_period

(Period('2010-01', 'M'), Period('2013-01', 'M'))

### Return

Next thing we derive from the `Close` values is [the `Return`](https://okama.io/#/glossary?page=cumulative-return). The `get_return` produces `TimeSeries` object that has one-month shorter `start_period`

In [11]:
assert asset.get_return().start_period == asset.close().start_period + 1
asset.get_return()

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.DIFF, values=[ 3.11904040e-02  6.08833798e-02  1.54926808e-02 -7.94811513e-02
 -5.17311395e-02  6.83006763e-02 -4.49818010e-02  8.95548276e-02
  3.82060559e-02  1.94222593e-05  6.68306964e-02  2.32950098e-02
  3.47378997e-02  1.24879193e-04  2.89620216e-02 -1.12151176e-02
 -1.68765302e-02 -2.00020986e-02 -5.49746532e-02 -6.94237430e-02
  1.09149744e-01 -4.06443285e-03  1.04519718e-02  4.63715156e-02
  4.34052335e-02  3.21615441e-02 -6.67481095e-03 -6.00569395e-02
  4.06257597e-02  1.17911078e-02  2.50526067e-02  2.53520427e-02
 -1.81964287e-02  5.69311471e-03  8.89824963e-03  5.11926024e-02]

`get_return` accepts parameters as follows:

- `kind: str` that determines the time series reduction. It should be one of `values` (by default), `cumulative` or `ytd` values
- `real: bool` that states if the return should be adjusted to corresponding inflation (`False` by default)

For example, the call as follows gets the real cumulative return for the period:

In [12]:
asset.get_return(kind='cumulative', real=True)

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.CUMULATIVE, values=[ 0.0309337   0.08922806  0.10418515  0.01563613 -0.03596299  0.02966413
 -0.01800814  0.06931163  0.10878537  0.10834029  0.18038355  0.20215473
  0.23781141  0.22601116  0.25344797  0.2335876   0.21406919  0.18873205
  0.12029215  0.04093713  0.15694192  0.15321292  0.16814809  0.21696226
  0.26421843  0.29504181  0.28252315  0.20691444  0.25779018  0.27469868
  0.29940203  0.32642601  0.30279658  0.3164509   0.33175141  0.39579985]

### CAGR

Asset supports computation of [CAGR](https://okama.io/#/glossary?page=compound-annual-growth-rate) activity:

In [13]:
asset.cagr()

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.REDUCED_VALUE, values=[0.14046465]

The asset can also compute the real CAGR:

In [14]:
asset.cagr(real=True)

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.REDUCED_VALUE, values=[0.11756909]

### Risk

The Risk ([Standard Deviation](https://okama.io/#/glossary?page=standard-deviation)) is computed with `risk: TimeSeries` method. It accepts `period` parameter that is either `year` (by default) or `month`:

In [15]:
asset.risk()

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.REDUCED_VALUE, values=[0.17183847]

In [16]:
asset.risk(period='year')

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.REDUCED_VALUE, values=[0.17183847]

In [17]:
asset.risk(period='month')

TimeSeries(start_period=2010-02, end_period=2013-01, kind=TimeSeriesKind.REDUCED_VALUE, values=[0.04331195]